In [ ]:
# First install LangChain
!pip install langchain


In [ ]:
# Environment setup

# Install OpenAI model APIs
!pip install OpenAI

# Store your OpenAI & SerpApi API keys in local variables, you could also store them as enviroment variables
OPENAI_API_KEY="YOUR_OPEN_API_KEY"
SERP_API_KEY = "YOUR_SERP_API_KEY"

In [ ]:
# LLMs

from langchain.llms import OpenAI

# Create and initialize an OpenAI model wrapper, pass in your OpenAI APi key as a parameter.
# In this case we want our outputs to be MORE random, so we will assign a HIGH temperature value.
llm = OpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.9)


In [ ]:
# Now we can pass in text and get some predictions
llm.predict("What would be a good company name for a company that makes interesting widgets?")

In [ ]:
# Chat models

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0)
# There are several types of messages currently supported by LangChain, for example AIMessage, HumanMessage, and SystemMessage.
chat.predict_messages([HumanMessage(content="Translate this sentence from English to Spanish. Language models are cool!")])

In [ ]:
# Prompt templates

from langchain.prompts import PromptTemplate

# In our earlier example, we asked the LLM to come up with a company name. Suppose now, we wanted to create a generic prompt template
# that could be used as part of an LLM application to come with a company name for whatever product a user chooses.
# We can easily do this by creating a reusable prompt template for that purpose:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="interesting widgets")

In [ ]:
# Chains
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0)
# Define a tamplate for the base prompt
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
# Chains provide way to link (or chain) together multiple building block, like models, prompts and even other chains!
chain = LLMChain(llm=chat, prompt=chat_prompt)
# Run the chain that passes values to the placholders in each prompt
chain.run(input_language="English", output_language="Spanish", text="Language models are cool!")

In [ ]:
# Agents

# To load an agent we will need select a(n):
# 1. LLM/Chat model: The underlying language model powering the agent
# 2. Tool(s): Some function that perfoms a duty. This could be things like: Google Search, Databse lookup, IFTTT webhooks and other chains. For
# the complete list see the  Tools documentation
# 3. Agent name: This is a text string that refers to a supported agent class. We're going to use one of the standard supported agents, but if you want to dig deeper,
# It is possible to implement custom agents for your purposes

# For this example we're going to use an Agent with Chat model that will leverage the Google Search results API to query a search engine.

# install the SerpAPI Python package:
!pip install google-search-results


In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(openai_api_key=OPENAI_API_KEY,temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm,serpapi_api_key=SERP_API_KEY)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is the founder of OpenAI? What is the latest news headline about him?")

In [ ]:
# Memory

# The memory module in LangChain provides a way to maintain application state. There are a number of built-in memory systems.
# The most simple of these is a buffer memory which just adds the last few inputs or outputs to the beginning of the current input - as shown in the example below:

from langchain import OpenAI, ConversationChain

llm = OpenAI(openai_api_key=OPENAI_API_KEY,temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.run("Hi there!")

In [ ]:

# Running the example again, you will see the model contains the input and output from the intial run.
conversation.run("I'm doing well! My name is Omer!")